In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('datasets/toyota.csv')
df

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,GT86,2016,16000,Manual,24089,Petrol,265,36.2,2.0
1,GT86,2017,15995,Manual,18615,Petrol,145,36.2,2.0
2,GT86,2015,13998,Manual,27469,Petrol,265,36.2,2.0
3,GT86,2017,18998,Manual,14736,Petrol,150,36.2,2.0
4,GT86,2017,17498,Manual,36284,Petrol,145,36.2,2.0
...,...,...,...,...,...,...,...,...,...
6733,IQ,2011,5500,Automatic,30000,Petrol,20,58.9,1.0
6734,Urban Cruiser,2011,4985,Manual,36154,Petrol,125,50.4,1.3
6735,Urban Cruiser,2012,4995,Manual,46000,Diesel,125,57.6,1.4
6736,Urban Cruiser,2011,3995,Manual,60700,Petrol,125,50.4,1.3


In [3]:
df['model'] = df['model'].str.strip()

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6738 entries, 0 to 6737
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         6738 non-null   object 
 1   year          6738 non-null   int64  
 2   price         6738 non-null   int64  
 3   transmission  6738 non-null   object 
 4   mileage       6738 non-null   int64  
 5   fuelType      6738 non-null   object 
 6   tax           6738 non-null   int64  
 7   mpg           6738 non-null   float64
 8   engineSize    6738 non-null   float64
dtypes: float64(2), int64(4), object(3)
memory usage: 473.9+ KB


In [5]:
df.isnull().sum()

model           0
year            0
price           0
transmission    0
mileage         0
fuelType        0
tax             0
mpg             0
engineSize      0
dtype: int64

## Importing ML Libraries

In [6]:
import numpy as np
import sklearn.metrics as m
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
# cross validation
from sklearn.model_selection import cross_val_score
# grid search
from sklearn.model_selection import GridSearchCV
# outlier
from sklearn.neighbors import LocalOutlierFactor
from sklearn.compose import ColumnTransformer

## Feature Selection

In [7]:
X = df.drop(columns=['price'])
y = df['price']

In [8]:
X 

,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,GT86,2016,Manual,24089,Petrol,265,36.2,2.0
1,GT86,2017,Manual,18615,Petrol,145,36.2,2.0
2,GT86,2015,Manual,27469,Petrol,265,36.2,2.0
3,GT86,2017,Manual,14736,Petrol,150,36.2,2.0
4,GT86,2017,Manual,36284,Petrol,145,36.2,2.0
...,...,...,...,...,...,...,...,...
6733,IQ,2011,Automatic,30000,Petrol,20,58.9,1.0
6734,Urban Cruiser,2011,Manual,36154,Petrol,125,50.4,1.3
6735,Urban Cruiser,2012,Manual,46000,Diesel,125,57.6,1.4
6736,Urban Cruiser,2011,Manual,60700,Petrol,125,50.4,1.3


## Column Extraction

In [9]:
bin_cols = ['transmission', 'fuelType']
num_cols = ['year', 'mileage', 'tax', 'mpg', 'engineSize']
cat_cols = ['model']

## Preprocessing Pipeline

In [10]:
# create pipeline
num_pipeline = Pipeline([
    ('std_scaler', StandardScaler())
])
binary_pipeline = Pipeline([
    ('one_hot_encoder', OrdinalEncoder())
])
cat_pipeline = Pipeline([
    ('one_hot_encoder', OneHotEncoder(drop='first'))
])

# create column transformer
preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_cols),
    ('binary', binary_pipeline, bin_cols),
    ('cat', cat_pipeline, cat_cols)
])

preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('std_scaler',
                                                  StandardScaler())]),
                                 ['year', 'mileage', 'tax', 'mpg',
                                  'engineSize']),
                                ('binary',
                                 Pipeline(steps=[('one_hot_encoder',
                                                  OrdinalEncoder())]),
                                 ['transmission', 'fuelType']),
                                ('cat',
                                 Pipeline(steps=[('one_hot_encoder',
                                                  OneHotEncoder(drop='first'))]),
                                 ['model'])])

## Outlier Removal
- this help to remove the outliers from the dataset

In [11]:
outlier_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('outlier', LocalOutlierFactor(n_neighbors=20, contamination=0.1))
])

outlier_pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('std_scaler',
                                                                   StandardScaler())]),
                                                  ['year', 'mileage', 'tax',
                                                   'mpg', 'engineSize']),
                                                 ('binary',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OrdinalEncoder())]),
                                                  ['transmission', 'fuelType']),
                                                 ('cat',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OneHotEncoder(drop='first'))]),
                                                  ['model'])])),
                ('outlier', LocalOutlierFactor(contamination=0.1))])

In [12]:
yhat = outlier_pipeline.fit_predict(X)
X = X[yhat==1].copy()
y = y[yhat==1].copy()

In [13]:
df = pd.concat([X, y], axis=1)

In [14]:
X# cleane data

,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,GT86,2016,Manual,24089,Petrol,265,36.2,2.0
1,GT86,2017,Manual,18615,Petrol,145,36.2,2.0
2,GT86,2015,Manual,27469,Petrol,265,36.2,2.0
3,GT86,2017,Manual,14736,Petrol,150,36.2,2.0
4,GT86,2017,Manual,36284,Petrol,145,36.2,2.0
...,...,...,...,...,...,...,...,...
6723,Verso-S,2013,Manual,33900,Petrol,125,51.4,1.3
6726,IQ,2009,Automatic,27292,Petrol,20,60.1,1.0
6727,IQ,2009,Manual,43000,Petrol,0,65.7,1.0
6728,IQ,2009,Manual,74095,Petrol,0,65.7,1.0


## Model Pipeline

In [15]:
pipeline1 = Pipeline([
    ('preprocessor', preprocessor),
    ('model', DecisionTreeRegressor())
])
pipeline1

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('std_scaler',
                                                                   StandardScaler())]),
                                                  ['year', 'mileage', 'tax',
                                                   'mpg', 'engineSize']),
                                                 ('binary',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OrdinalEncoder())]),
                                                  ['transmission', 'fuelType']),
                                                 ('cat',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OneHotEncoder(drop='first'))]),
                                                  ['model'])])),
                ('model', DecisionTreeRegressor())])

In [16]:
pipeline2 = Pipeline([
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor())
])
pipeline2

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('std_scaler',
                                                                   StandardScaler())]),
                                                  ['year', 'mileage', 'tax',
                                                   'mpg', 'engineSize']),
                                                 ('binary',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OrdinalEncoder())]),
                                                  ['transmission', 'fuelType']),
                                                 ('cat',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OneHotEncoder(drop='first'))]),
                                                  ['model'])])),
                ('model', RandomForestRegressor())])

In [17]:
pipeline3 = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LinearRegression())
])
pipeline3

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('std_scaler',
                                                                   StandardScaler())]),
                                                  ['year', 'mileage', 'tax',
                                                   'mpg', 'engineSize']),
                                                 ('binary',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OrdinalEncoder())]),
                                                  ['transmission', 'fuelType']),
                                                 ('cat',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OneHotEncoder(drop='first'))]),
                                                  ['model'])])),
                ('model', LinearRegression())])

In [18]:
pipeline4 = Pipeline([
    ('preprocessor', preprocessor),
    ('model', SVR())
])
pipeline4

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('std_scaler',
                                                                   StandardScaler())]),
                                                  ['year', 'mileage', 'tax',
                                                   'mpg', 'engineSize']),
                                                 ('binary',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OrdinalEncoder())]),
                                                  ['transmission', 'fuelType']),
                                                 ('cat',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OneHotEncoder(drop='first'))]),
                                                  ['model'])])),
                ('model', SVR())])

## Training Model and Evaluation

In [19]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)
model1 = pipeline1.fit(train_X, train_y)
yhat = model1.predict(test_X)
mae = m.mean_absolute_error(test_y, yhat)
mse = m.mean_squared_error(test_y, yhat)
r2 = m.r2_score(test_y, yhat)
rmse = np.sqrt(mse)
print("Decision Tree Regressor")
print(f'MAE: {mae} - tells us how far off our predictions are on average.')
print(f'MSE: {mse} - tells us how far off our predictions are on average squared.')
print(f'RMSE: {rmse} - tells us how far off our predictions are on average squared root.')
print(f'R2: {r2} - tells us how much of the variance in the target variable is explained by the model.')

Decision Tree Regressor
MAE: 972.3347073371806 - tells us how far off our predictions are on average.
MSE: 2097997.937895026 - tells us how far off our predictions are on average squared.
RMSE: 1448.4467328469577 - tells us how far off our predictions are on average squared root.
R2: 0.9316186556549237 - tells us how much of the variance in the target variable is explained by the model.


In [20]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)
model2 = pipeline2.fit(train_X, train_y)
yhat = model2.predict(test_X)
mae = m.mean_absolute_error(test_y, yhat)
mse = m.mean_squared_error(test_y, yhat)
r2 = m.r2_score(test_y, yhat)
rmse = np.sqrt(mse)
print("Random Forest Regressor")
print(f'MAE: {mae} - tells us how far off our predictions are on average.')
print(f'MSE: {mse} - tells us how far off our predictions are on average squared.')
print(f'RMSE: {rmse} - tells us how far off our predictions are on average squared root.')
print(f'R2: {r2} - tells us how much of the variance in the target variable is explained by the model.')

Random Forest Regressor
MAE: 790.7389101499626 - tells us how far off our predictions are on average.
MSE: 1271428.155668739 - tells us how far off our predictions are on average squared.
RMSE: 1127.5762305355408 - tells us how far off our predictions are on average squared root.
R2: 0.958559555778191 - tells us how much of the variance in the target variable is explained by the model.


In [21]:
model3 = pipeline3.fit(train_X, train_y)
yhat = model3.predict(test_X)
mae = m.mean_absolute_error(test_y, yhat)
mse = m.mean_squared_error(test_y, yhat)
r2 = m.r2_score(test_y, yhat)
rmse = np.sqrt(mse)
print("Linear Regression")
print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'R2: {r2}')

Linear Regression
MAE: 1151.1891039497532
MSE: 2797027.552273412
RMSE: 1672.4316285795996
R2: 0.9088347511024848


In [22]:
model4 = pipeline4.fit(train_X, train_y)
yhat = model4.predict(test_X)
mae = m.mean_absolute_error(test_y, yhat)
mse = m.mean_squared_error(test_y, yhat)
r2 = m.r2_score(test_y, yhat)
rmse = np.sqrt(mse)
print("Support Vector Regression")
print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'R2: {r2}')

Support Vector Regression
MAE: 3810.950215723507
MSE: 30781751.46852139
RMSE: 5548.130448044763
R2: -0.0032886632984077124


In [23]:
df['dtree_pred'] = model1.predict(df.drop(columns=['price']))
df['rf_pred'] = model2.predict(df.drop(columns=['price']))
df['lr_pred'] = model3.predict(df.drop(columns=['price']))
df['svr_pred'] = model4.predict(df.drop(columns=['price']))

## visualization

In [24]:
# histogram
fig = px.histogram(df, x='price', nbins=100, title='Price Distribution')
fig.show()

## Saving the model

In [25]:
import joblib

In [26]:
joblib.dump(model1, 'toyota/dtree_model.pkl')
joblib.dump(model2, 'toyota/rf_model.pkl')
joblib.dump(model3, 'toyota/lr_model.pkl')
joblib.dump(model4, 'toyota/svr_model.pkl')

['toyota/svr_model.pkl']

In [27]:
X.fuelType.unique()

array(['Petrol', 'Hybrid', 'Diesel', 'Other'], dtype=object)

In [28]:
X.model.unique()

array(['GT86', 'Corolla', 'RAV4', 'Yaris', 'Auris', 'Aygo', 'C-HR',
       'Prius', 'Avensis', 'Verso', 'Hilux', 'Land Cruiser', 'Verso-S',
       'IQ'], dtype=object)

In [29]:
X.transmission.unique()

array(['Manual', 'Automatic', 'Semi-Auto'], dtype=object)

In [30]:
X

,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,GT86,2016,Manual,24089,Petrol,265,36.2,2.0
1,GT86,2017,Manual,18615,Petrol,145,36.2,2.0
2,GT86,2015,Manual,27469,Petrol,265,36.2,2.0
3,GT86,2017,Manual,14736,Petrol,150,36.2,2.0
4,GT86,2017,Manual,36284,Petrol,145,36.2,2.0
...,...,...,...,...,...,...,...,...
6723,Verso-S,2013,Manual,33900,Petrol,125,51.4,1.3
6726,IQ,2009,Automatic,27292,Petrol,20,60.1,1.0
6727,IQ,2009,Manual,43000,Petrol,0,65.7,1.0
6728,IQ,2009,Manual,74095,Petrol,0,65.7,1.0
